In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
os.getcwd()

'/scratch/anissa.el/data_analysis/notebooks/karen_building_blocks_analysis'

In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.optimize import minimize,LinearConstraint, SR1
from qpsolvers import solve_qp
from src.utils.archetypes import ArchetypalAnalysis
from src.CellAbundance import CellAbundance, generate_abundance_matrix, join_abundance_matrices
from src.utils.visualization import plot_scatter_pca, plot_cumulative_explained_variance, plot_PCAs, plot_CVEs

In [3]:
##########################################################
#      CONTROL PANEL       #  set your parameters here   #
##########################################################
patient_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 
               26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]
N_SITE = 100
RADIUS = 25
# RADIUS = 25
#METHOD = "abs"#'gaussian' 
METHOD = 'gaussian'
CELL_TYPES = ['CD8-T', 'Other immune', 'DC / Mono', 'CD3-T', 'B', 'NK', 'Keratin-positive tumor', 'Tumor', 
              'CD4-T', 'Mesenchymal-like', 'Macrophages', 'Endothelial', 'Tregs', 'Unidentified', 'DC', 'Mono / Neu', 
              'Neutrophils']
ROOT_DATA_PATH = "../../data/cell_positions_data"#"../../../../fabio/output/cell_positions_data"
ROOT_OUTPUT_CSV_PATH = "../../output"
RANDOM_SEED = 1022

In [4]:
# Generating the sites and cell abundance!
abundance_matrix_center_cells = generate_abundance_matrix(CELL_TYPES, patient_ids, N_SITE, RADIUS, METHOD,center_sites_cells=True, root=ROOT_DATA_PATH, random_seed=RANDOM_SEED)

In [5]:
# Generating the sites and cell abundance!
abundance_matrix = generate_abundance_matrix(CELL_TYPES, patient_ids, N_SITE, RADIUS, METHOD,center_sites_cells=False, root=ROOT_DATA_PATH, random_seed=RANDOM_SEED)

In [19]:
# generating the dataset with all the sites from the patients and the cells belonging to each site centered by cell (point one)

df = pd.DataFrame()
for ca in abundance_matrix_center_cells:
    df_ca = ca.get_site_cell_id_df()#get_site_cell_map_dataframe()
    df_ca['patient_id'] = int(ca.patient_id)
    df = df.append(df_ca)
df = df.reset_index(drop = True)
'''
df = [] #pd.DataFrame()
for ca in abundance_matrix_center_cells:
    df_ca = ca.get_site_cell_map_dataframe()
    df_ca['patient_id'] = int(ca.patient_id)
    df.append(df_ca)
df = pd.concat(df)
#df = df.reset_index(drop = True)
'''
df
#df.to_csv("/srv/mfs/hausserlab/anissa.el/ImmuneStates/data_BB/site_cells_gaussian.csv", index=False, header=True)#df.to_csv("{}/site_cells.csv".format(ROOT_OUTPUT_CSV_PATH), index=False, header=True)

,cell_id,cell_type,patient_id
0,2,Keratin-positive tumor,1
1,3,CD3-T,1
2,4,Keratin-positive tumor,1
3,5,Keratin-positive tumor,1
4,6,B,1
...,...,...,...
197673,5093,Keratin-positive tumor,41
197674,5094,Tumor,41
197675,5095,Macrophages,41
197676,5096,Other immune,41


In [20]:
## Export dataframe in csv format
df
df.to_csv("/scratch/anissa.el/ImmuneStates/data_BB/site_centered_cells.csv", index=False, header=True)#df.to_csv("{}/site_cells.csv".format(ROOT_OUTPUT_CSV_PATH), index=False, header=True)

In [6]:
#Abundance Matrix 
df_2 = pd.DataFrame() 
for ca in abundance_matrix:
    abundance_df = pd.DataFrame(ca.abundance_matrix,columns = CELL_TYPES)
    abundance_df['site_id'] = np.arange(len(abundance_df))
    abundance_df['patient_id'] = ca.patient_id
    df_2 = df_2.append(abundance_df)
    
df_2 = df_2.reset_index()


In [7]:
#df_2
print(df_2.shape)
#print(df_2.iloc[:,1:17])
#np.mean(np.mean(np.array(df_2.iloc[:,1:17]),axis=1),axis=0)
#df_2.to_csv("/srv/mfs/hausserlab/anissa.el/ImmuneStates/data_BB/abundance_matrix_gaussian.csv", index=False, header=True)

(4000, 20)


In [7]:
# Archetypal Analysis
sites, patients_ids,sites_ids,gradients = join_abundance_matrices(abundance_matrix)
pca = PCA()
pc = pca.fit_transform(sites)
pca3d = pd.DataFrame(pc[:, :3], columns = ['PC1', 'PC2', 'PC3'])
pca3d['site_id'] = df_2['site_id']
pca3d['patient_id'] = df_2['patient_id']
#pca3d.to_csv("/srv/mfs/hausserlab/anissa.el/ImmuneStates/data_BB/sites_pca_matrix_gaussian.csv", index=False, header=True)
AA_3D = ArchetypalAnalysis(n_archetypes = 4, 
                        tolerance = 0.001, 
                        max_iter = 200, 
                        random_state = 0, 
                        C = 0.0001, 
                        initialize = 'random',
                        redundancy_try = 30)
AA_3D.fit_transform(pc[:, :3])

array([[ 8.36960480e-04, -2.05139738e-04,  5.13431956e-04],
       [ 6.22451888e-04,  1.79993309e-04,  4.66145708e-04],
       [-5.45279156e-05, -1.05861330e-04,  5.32484013e-04],
       ...,
       [-1.11045895e-03, -8.26919234e-04,  1.67422287e-04],
       [-3.43905627e-03, -1.31344254e-03,  8.50245620e-05],
       [-2.79474995e-03, -9.07546280e-04, -2.04928635e-04]])

In [30]:
#open it
df_archetypes = pd.DataFrame(AA_3D.alfa.T, columns=['archetype1', 'archetype2', 'archetype3', 'archetype4'])
df_archetypes['site_id'] = df_2['site_id']
df_archetypes['patient_id'] = df_2['patient_id']


### Getting TMENs proportions of sites centered on cells

In [14]:
### Infer alfas of sites centered on cells (sites) from TMENs found in randomly generated sites (sites2)
n_comp = 3
sites2, patients_ids2,sites_ids2,gradients2 = join_abundance_matrices(abundance_matrix_center_cells)
tmens = np.dot(AA_3D.archetypes.T, pca.components_[:n_comp,:])+np.mean(sites, axis=0)
#print(tmens.shape)
#print(sites2.T.shape)
# C (n x m, n= nb of cell types, m= nb of sites): cellular abundance of sites centered by cells
# B (n x t, t = nb of TMENs): cell abundance of TMENs found in random sites
# alfa (t x m): Prop of TMENs for each sites centered by cells

## FIXME: minimizing function to get alfas that respect constraints of proportions
def f(x,*args):
    site, tmens = args
    x = np.expand_dims(x, 1)
    #print(x.T.shape)
    return np.linalg.norm(site - np.dot(x.T,tmens))
     
#m: number of sites
#n: number of cell types
#N=4: number of TMENs
print(sites2.shape[0])
print(tmens.shape)
x0 = np.array([[0.5,0.05,0.4,0.05]]) #np.zeros((4,sites2.shape[0]))


# Constraints of proportions: non-negative values, sum equal to 1
constr = ({'type':'eq', 'fun': lambda x: sum(x) - 1,'tol':0.001},
         {'type': 'ineq', 'fun': lambda x: x,'tol':0.001},
         {'type': 'ineq','fun': lambda x: 1-x,'tol':0.001})#lambda x: np.array([x[:,:],x[:,0] +  x[:,1] + x[:,2]+ x[:,3]- 1 ])


NB_TMENS = tmens.shape[0]
sites_alfas = np.zeros(shape = (sites2.shape[0],NB_TMENS))
for i in np.arange(sites2.shape[0]):
    site = sites2[i,:].T
    #print(sites_ids2[i])
    P = np.dot(tmens, tmens.T)
    q = -np.dot(tmens,site)
    A = np.ones(NB_TMENS)
    b = 1
    G = np.diag(-np.ones(NB_TMENS))
    h = np.zeros(NB_TMENS)
    #print(A)
    res = solve_qp(P, q, G, h, A, b)
    sites_alfas[i] = res
    #print("QP solution: x = {}".format(res))
print(np.sum(sites_alfas[0:9,:],axis = 1))

197678
(4, 17)
[1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [16]:
sites_archs = pd.DataFrame(sites_alfas, columns = ["arch1","arch2","arch3","arch4"])
sites_archs['patient_id'] = patients_ids2
sites_archs["site_id"] = sites_ids2[:,0]
sites_archs["cell_type_site"] = sites_ids2[:,1]
#print(sites_archs)
sites_archs.to_csv("/scratch/anissa.el/ImmuneStates/data_BB/archetypes_sites_centered_cells_gaussian.csv",index=False,header=True)

In [22]:
#df_archetypes2.to_csv("/scratch/anissa.el/ImmuneStates/data_BB/archetypes_sites_centered_cells_gaussian.csv", index=False, header=True)

In [32]:
df_archetypes
#df_archetypes.to_csv("/srv/mfs/hausserlab/anissa.el/ImmuneStates/data_BB/archetypes_matrix_gaussian.csv", index=False, header=True)

In [23]:
#FIXME : re-compute the new cell densities from each archetype with GAUSSIAN counting ==> consistency !!

n_comp = 3
res = np.dot(AA_3D.archetypes.T, pca.components_[:n_comp,:])+np.mean(sites, axis=0)
archetypes_cell_space = pd.DataFrame(res,  columns = CELL_TYPES)
archetypes_cell_space["archetype"] = ['ARCH1', 'ARCH2', 'ARCH3', 'ARCH4']
archetypes_cell_space

,CD8-T,Other immune,DC / Mono,CD3-T,B,NK,Keratin-positive tumor,Tumor,CD4-T,Mesenchymal-like,Macrophages,Endothelial,Tregs,Unidentified,DC,Mono / Neu,Neutrophils,archetype
0,-0.001750,-0.000236,0.000082,-0.000005,0.013639,-0.000003,-0.000369,0.000106,0.005125,0.000288,-0.002278,0.000020,-0.000121,0.000022,0.000176,-0.000253,-0.000047,ARCH1
1,0.006750,0.000587,0.001184,0.000768,-0.001002,0.000393,0.000359,-0.000257,0.006473,-0.000456,0.003933,0.000154,0.000437,-0.000095,0.000174,0.000242,0.000211,ARCH2
2,0.000017,-0.000375,-0.000140,-0.000084,-0.000092,-0.000011,0.010285,0.000116,-0.000131,-0.000210,-0.000242,-0.000092,0.000044,-0.000040,-0.000041,0.000007,-0.000015,ARCH3
3,0.000130,0.000605,0.000248,0.000184,0.000030,-0.000003,-0.000131,0.000158,-0.000272,0.000718,0.001126,0.000180,0.000007,0.000141,0.000053,0.000183,0.000189,ARCH4


In [25]:
#TODO: set column and row names when converting dataframe to csv file!

archetypes_cell_space.to_csv("/scratch/anissa.el/ImmuneStates/data_BB/archetypes_building_blocks_17cells_space_gaussian.csv",index=False,header=True)
